In [1]:
import re
import time

In [2]:
with open("input_07.txt", "r") as fh:
    content = fh.readlines()

In [3]:
def get_instructions(content):
    instructions = {}
    for c in content:
        i, o = c.split("->")
        o = o.strip()
        result = re.findall(r"\w+", i)
        assert o not in instructions, "Output expected only once"
        if len(result) == 1:
            instructions[o] = (None, result[0])
        elif len(result) == 2:
            assert result[0] == "NOT", "NOT operator was expected"
            instructions[o] = (result[0], result[1])
        elif len(result) == 3:
            instructions[o] = (result[1], result[0], result[2])
        else:
            print("Error: this should never happen.")
    return instructions


def parse(instructions):
    outputs = {}
    while len(instructions):
        remove_keys = []
        for k in instructions:
            i = instructions[k]
            match i[0]:
                case None:
                    if i[1].isdigit():
                        outputs[k] = int(i[1])
                        remove_keys.append(k)
                    elif i[1] in outputs:
                        outputs[k] = outputs[i[1]]
                        remove_keys.append(k)
                case "NOT":
                    if i[1].isdigit():
                        outputs[k] = ~int(i[1])
                        remove_keys.append(k)
                    elif i[1] in outputs:
                        outputs[k] = ~outputs[i[1]]
                        remove_keys.append(k)
                case _:
                    method = i[0]
                    operand1, operand2 = None, None
                    if i[1].isdigit():
                        operand1 = int(i[1])
                    elif i[1] in outputs:
                        operand1 = outputs[i[1]]
                    if i[2].isdigit():
                        operand2 = int(i[2])
                    elif i[2] in outputs:
                        operand2 = outputs[i[2]]
                    if operand1 is not None and operand2 is not None:
                        remove_keys.append(k)
                        match method:
                            case "AND":
                                outputs[k] = operand1 & operand2
                            case "OR":
                                outputs[k] = operand1 | operand2
                            case "LSHIFT":
                                outputs[k] = operand1 << operand2
                            case "RSHIFT":
                                outputs[k] = operand1 >> operand2
                            case _:
                                print("Error: this should never happen.")
        # Remove keys from instructions
        for k in remove_keys:
            del instructions[k]

        # Convert signed integers to unsigned equivalent (i.e., most significant bit should not be the sign)
    for o in outputs:
        outputs[o] = (outputs[o] + 2**16) % 2**16
    return outputs

In [4]:
# 7a
t0 = time.time()
instructions = get_instructions(content)
outputs = parse(instructions)
print("7a:", outputs["a"], f"in {time.time() - t0} sec.")

7a: 3176 in 0.002394437789916992 sec.


In [5]:
# 7b
instructions = get_instructions(content)
instructions["b"] = (None, str(outputs["a"]))
outputs = parse(instructions)
print("7b:", outputs["a"], f"in {time.time() - t0} sec.")

7b: 14710 in 0.010710000991821289 sec.
